In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# from pycaret.regression import *

df_train = pd.read_csv('../input/bike-sharing-demand/train.csv')
df_test = pd.read_csv('../input/bike-sharing-demand/test.csv')
df_train.head()

In [ ]:
df_train['datetime'] = df_train['datetime'].astype('datetime64[ns]')
df_train['day'] = df_train.datetime.dt.day
df_train['month'] = df_train.datetime.dt.month
df_train['year'] = df_train.datetime.dt.year
df_train['hour'] = df_train.datetime.dt.hour

df_test['datetime'] = df_test['datetime'].astype('datetime64[ns]')
df_test['day'] = df_test.datetime.dt.day
df_test['month'] = df_test.datetime.dt.month
df_test['year'] = df_test.datetime.dt.year
df_test['hour'] = df_test.datetime.dt.hour
#df_test[['day','month','year']] = df_test.datetime.str.split('-',expand=True).astype(int)
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train['day_of_week'] = df_train['datetime'].dt.day_name()
df_train['day_of_week'].value_counts()

In [ ]:
plt.figure(figsize = [20, 10])
base_color = sns.color_palette()[0]
_ = sns.boxplot(data=df_train, x='day_of_week', y='count', color=base_color)

**Week days alomst the same in the numbers of bikes so it may be not important feature**

In [ ]:
plt.figure(figsize = [30, 5])
# sns.set(rc={'figure.figsize': [20, 10]}, font_scale=.8)
sns.jointplot(x='humidity', y='count', data=df_train, kind='scatter',alpha= 0.1)

**We can notich that the relation between humidity and the count of bikes is week**

In [ ]:
plt.figure(figsize = [20, 10])
base_color = sns.color_palette()[0]
 
plt.subplot(1, 2, 1)
# plot sex VS total_bill
_ = sns.scatterplot(data=df_train, x='windspeed', y='count', color=base_color)

plt.subplot(1, 2, 2)
# plot sex VS tip
_ = sns.scatterplot(data=df_train, x='weather', y='count', color=base_color)

**We can notice the wind speed have strong relation with the count of bikes but the same with weather feature**

In [ ]:
plt.figure(figsize = [40, 10])
base_color = sns.color_palette()[0]
_ = sns.boxplot(data=df_train, x='temp', y='count', color=base_color)

In [ ]:
plt.figure(figsize = [40, 30])
base_color = sns.color_palette()[0]
 
plt.subplot(2, 3, 1)
# plot sex VS total_bill
_ = sns.boxplot(data=df_train, x='workingday', y='count', color=base_color)

plt.subplot(2,3, 2)
# plot sex VS tip
_ = sns.boxplot(data=df_train, x='holiday', y='count', color=base_color)

plt.subplot(2, 3, 3)
# plot sex VS tip
_ = sns.boxplot(data=df_train, x='year', y='count', color=base_color)

#row 2
plt.subplot(2, 3, 4)
# plot sex VS total_bill
_ = sns.stripplot(data=df_train, x='workingday', y='count', color=base_color)

plt.subplot(2, 3, 5)
# plot sex VS tip
_ = sns.stripplot(data=df_train, x='holiday', y='count', color=base_color)

plt.subplot(2, 3, 6)
# plot sex VS tip
_ = sns.stripplot(data=df_train, x='year', y='count', color=base_color)

In [ ]:
df_train.info()

In [ ]:
df_train.drop(columns=['day_of_week','datetime','casual','registered'],axis=1,inplace = True)
df_test.drop(columns=['datetime'],axis=1,inplace = True)
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train['windspeed'] = df_train['windspeed'].mask((df_train['windspeed'] > 10) & (df_train['windspeed'] <= 20) , 15)
df_train['windspeed'] = df_train['windspeed'].mask((df_train['windspeed'] > 20) & (df_train['windspeed'] <= 28) , 24)
df_train['windspeed'] = df_train['windspeed'].mask(df_train['windspeed'] > 28 , 30)

df_test['windspeed'] = df_test['windspeed'].mask((df_test['windspeed'] > 10) & (df_test['windspeed'] <= 20) , 15)
df_test['windspeed'] = df_test['windspeed'].mask((df_test['windspeed'] > 20) & (df_test['windspeed'] <= 28) , 24)
df_test['windspeed'] = df_test['windspeed'].mask(df_test['windspeed'] > 28 , 30)
print(df_train['windspeed'].value_counts())

In [ ]:
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score
# mean_squared_log_error(y_val, y_pred)

y = df_train['count']            
x = df_train.drop(['count'], axis=1)
# print(x.info())
y.shape
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x , y , train_size=0.8 , test_size=0.2,random_state=0)
y_val.shape

In [ ]:
df_train.info()

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
et_model = ExtraTreesRegressor(n_estimators=100, random_state=0,criterion='mae').fit(x_train, y_train)
y_pred_et = et_model.predict(x_val)
print("RMSLE is :",np.sqrt(mean_squared_log_error(abs(y_val), abs(y_pred_et))))
print("R2 score is :",r2_score(y_val, y_pred_et))

In [ ]:
y_final_pred = et_model.predict(df_test)

submission = pd.read_csv("../input/bike-sharing-demand/test.csv")
submission_df = pd.DataFrame()
submission_df["datetime"] = submission.datetime

submission_df["count"] = y_final_pred
submission_df.head(10)

In [ ]:
submission_df.to_csv("submission1.csv", index = False, header = True)
